#Setup

##Imports

In [19]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

##Data

###Load Data

In [2]:
text = list(pd.read_csv('/content/in_domain_train.tsv', sep='\t', header=None)[3]) + list(pd.read_csv('/content/in_domain_dev.tsv', sep='\t', header=None)[3]) + list(pd.read_csv('/content/out_of_domain_dev.tsv', sep='\t', header=None)[3])
text[-10: -1]

['John thinks it would upset himself to die.',
 'John made Bill mad at himself.',
 'John made Bill master of himself.',
 'The correspondence school made Bill a good typist.',
 'The correspondence school sent Bill a good typist.',
 'John considers Bill silly.',
 'John considers Bill to be silly.',
 'John bought a dog for himself to play with.',
 'John arranged for himself to get the prize.']

In [3]:
print(len(text))

9594


###Preprocess Text

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
tokenized_text = tokenizer.texts_to_sequences(text)
len(tokenized_text)

9594

In [5]:
idx_word = {v: k for k, v in tokenizer.word_index.items()}
print(idx_word)

In [6]:
# Making Sure That Every Element Has length > 1
for ele in tokenized_text:
    if len(ele) <= 1:
        print(ele)
        tokenized_text.remove(ele)


9594


In [7]:
# Randomly Picking The Missing Word(Output) and Using the rest of the Sentence as the Input.
# Using each sentence thrice
outputs = []
inputs = []
for i in range(len(tokenized_text)):
    seq1 = list(tokenized_text[i])
    seq2 = list(tokenized_text[i])
    seq3 = list(tokenized_text[i])
    ele1 = random.choice(seq1)
    ele2 = random.choice(seq2)
    ele3 = random.choice(seq3)
    outputs.append(ele1)
    outputs.append(ele2)
    outputs.append(ele3)
    seq1.remove(ele1)
    seq2.remove(ele2)
    seq3.remove(ele3)
    inputs.append(seq1)
    inputs.append(seq2)
    inputs.append(seq3)

idx = 2
print(inputs[idx])
print(outputs[idx])


[204, 289, 232, 163, 28, 777, 778, 1251, 779, 72, 25, 3828]
1


In [8]:
inp_padded = pad_sequences(inputs)
inp_padded.shape

(28782, 41)

#Build and Train Model

In [9]:
depth = int(list(tokenizer.word_index.values())[-1] + 1)

##Custom Layers

In [10]:
class PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, depth):
        super(PreprocessLayer, self).__init__()
        self.depth = depth
    
    def call(self, inputs):
        inp_one_hot = tf.one_hot(inputs, depth=self.depth)

        return inp_one_hot, inputs

class AveragePoolingLayer(tf.keras.layers.Layer):
    def __init__(self, ignore=0):
        super(AveragePoolingLayer, self).__init__()
        self.ignore = ignore
    
    def call(self, inp_one_hot, inp_padded):
        inp_pad_mask = tf.cast((inp_padded != self.ignore), tf.float64)
        inp_mask_sum = tf.math.reduce_sum(inp_pad_mask, axis=-1)
        inp_one_hot_1 = inp_one_hot  * tf.expand_dims(inp_pad_mask, axis=-1)
        inp_one_hot_avg = tf.math.reduce_sum(inp_one_hot_1, axis=1) / tf.expand_dims(inp_mask_sum, axis=-1)

        return inp_one_hot_avg


##Initialize The Model

In [11]:
DenseLayer1 = tf.keras.layers.Dense(256, activation='relu')
DenseLayer2 = tf.keras.layers.Dense(depth, activation='softmax')
model_inp = tf.keras.layers.Input(shape=(41,), dtype=tf.int32)
one_hot_inp, padded_inp = PreprocessLayer(depth=depth)(model_inp)
x = AveragePoolingLayer()(one_hot_inp, padded_inp)
x = DenseLayer1(x)
model_out = DenseLayer2(x)
model = tf.keras.Model(inputs=model_inp, outputs=model_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

##Train The Model

In [12]:
model.fit(inp_padded, np.array(outputs), epochs=40)

Epoch 1/40
900/900 [==============================] - 10s 8ms/step - loss: 6.7742 - accuracy: 0.0861
Epoch 2/40
900/900 [==============================] - 7s 8ms/step - loss: 6.2012 - accuracy: 0.0879
Epoch 3/40
900/900 [==============================] - 7s 8ms/step - loss: 5.9079 - accuracy: 0.1039
Epoch 4/40
900/900 [==============================] - 7s 8ms/step - loss: 5.5292 - accuracy: 0.1311
Epoch 5/40
900/900 [==============================] - 7s 8ms/step - loss: 5.0474 - accuracy: 0.1791
Epoch 6/40
900/900 [==============================] - 7s 8ms/step - loss: 4.4919 - accuracy: 0.2407
Epoch 7/40
900/900 [==============================] - 7s 8ms/step - loss: 3.8969 - accuracy: 0.3081
Epoch 8/40
900/900 [==============================] - 7s 8ms/step - loss: 3.3010 - accuracy: 0.3804
Epoch 9/40
900/900 [==============================] - 7s 8ms/step - loss: 2.7437 - accuracy: 0.4589
Epoch 10/40
900/900 [==============================] - 7s 8ms/step - loss: 2.2596 - accuracy: 0.534

##Predict using a random sentence
This is majorly done to check if the model works decently well on a test example. It is a very simple sanity check.

In [17]:
def predict(sentence):
    tokenized = tokenizer.texts_to_sequences([sentence])
    tokenized.append([0] * 41)
    tokenized = pad_sequences(tokenized)
    out = model.predict(tokenized)[0]
    max_out = tf.math.argmax(out)
    output = idx_word.get(max_out.numpy(), "")
    print(output)
predict("the cat sat on the")

table


In [18]:
predict("on sat the cat the")

table


We can see that the model predicts the same word even though the order of the words are shuffled. 

This is because the Continuous Bag Of Words(CBOW) model we use here doesn't care about the order of the words while predicting.

#Word Embeddings

##Extracting The Embeddings

In [80]:
weight_one = DenseLayer1.get_weights()[0]
weight_two = DenseLayer2.get_weights()[0]
embeddings = 0.5 * (weight_one + weight_two.T)
embeddings.shape

(5828, 256)

##Saving The Embeddings

###Saving as a .txt File

In [83]:
with open("/content/word2vecembed.txt", 'w') as file:
    for i in range(len(idx_word)):
        file.write(f"{idx_word[i+1]} {embeddings[i+1]}")

###Saving as a .tsv file
TSV files are better while saving word embeddings as TSVs are the file types suported in Embedding Projectors.

In [91]:
import io

In [92]:
out_v = io.open('word2vec_embeddings.tsv', 'w', encoding='utf-8')
out_m = io.open('word2vec_vocab.tsv', 'w', encoding='utf-8')

for index, word in enumerate(list(idx_word.values())):
    vec = embeddings[index+1]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

###Downloading The Files

In [93]:
try:
  from google.colab import files
  files.download('embeddings.tsv')
  files.download('words.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [94]:
try:
  from google.colab import files
  files.download('/content/word2vecembed.txt')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>